# Goal　目標
- Creating a gif animation of all sliced images with mask.<br>
スライス画像とマスク情報のGIFアニメーションを作成します。
- Please copy & edit this code, If you have a case-day you'd like create the animation.<br>
作りたいcase-dayのアニメーションがあれば、是非copy & editしてください。

If you'd like to get all case-days animation quickly, please refer to the dataset.<br>
手っ取り早く全てのアニメーションが見たい場合、以下のデータセットを参照ください。<br>
https://www.kaggle.com/datasets/utm529fg/umdgi-gif-animation-of-3d-mask-image

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import Image, display, HTML

from glob import glob
from PIL import Image
import os, shutil
import cv2

import tensorflow as tf
from tqdm import tqdm
tqdm.pandas()
from matplotlib.patches import Rectangle

# Prepare the Data　データ準備

ref https://www.kaggle.com/code/awsaf49/uwmgi-mask-data

In [ ]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = np.asarray(mask_rle.split(), dtype=int)
    starts = s[0::2] - 1
    lengths = s[1::2]
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape) 

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
def get_metadata(row):
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

def path2info(row):
    path = row['image_path']
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

In [ ]:
def id2mask(id_):
    idf = df_train[df_train['id']==id_]
    wh = idf[['height','width']].iloc[0]
    shape = (wh.height, wh.width, 3)
    mask = np.zeros(shape, dtype=np.uint8)
    for i, class_ in enumerate(['large_bowel', 'small_bowel', 'stomach']):
        cdf = idf[idf['class']==class_]
        rle = cdf.segmentation.squeeze()
        if len(cdf) and not pd.isna(rle):
            mask[..., i] = rle_decode(rle, shape[:2])
    return mask

def rgb2gray(mask):
    pad_mask = np.pad(mask, pad_width=[(0,0),(0,0),(1,0)])
    gray_mask = pad_mask.argmax(-1)
    return gray_mask

def gray2rgb(mask):
    rgb_mask = tf.keras.utils.to_categorical(mask, num_classes=4)
    return rgb_mask[..., 1:].astype(mask.dtype)

In [ ]:
def load_img(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = img.astype('float32') 
    img = (img - img.min())/(img.max() - img.min())*255.0 
    img = img.astype('uint8')
    return img

def show_img(img, mask=None):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    plt.figure(figsize=(10,10))
    plt.imshow(img, cmap='bone')
    
    if mask is not None:
        plt.imshow(mask, alpha=0.5)
        handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]
        labels = [ "Large Bowel", "Small Bowel", "Stomach"]
        plt.legend(handles,labels)
    plt.axis('off')

In [ ]:
df_train = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
df_train = df_train.progress_apply(get_metadata, axis=1)
paths = glob('../input/uw-madison-gi-tract-image-segmentation/train/*/*/*/*')
path_df = pd.DataFrame(paths, columns=['image_path'])
path_df = path_df.progress_apply(path2info, axis=1)
df_train = df_train.merge(path_df, on=['case','day','slice'])
df_case_day = df_train[['case','day']].drop_duplicates(['case','day']).sort_values(['case','day']).reset_index(drop=True)

# Setting of the Animation　アニメーションの設定

In [ ]:
def create_animation(case, day, file_name):
    print('drawing image...')
    TRAIN_DIR='../input/uw-madison-gi-tract-image-segmentation/train/case'+case+'/case'+case+'_day'+day+'/scans/'
    train_images = glob(os.path.join(TRAIN_DIR, '**', '*.png'), recursive=True)
    train_images = sorted(train_images)

    fig, ax = plt.subplots(figsize=(10, 10))
    ims = []
    
    # legend of mask
    handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]
    labels = ['Large Bowel', 'Small Bowel', 'Stomach']
    plt.legend(handles,labels)
    
    plt.axis('off')

    text1 = ax.text(0, 1.03, 'case '+case+' day '+day,
             ha='left', va='bottom',
             transform=ax.transAxes, fontsize='x-large',backgroundcolor='white')
    
    slice_i=1
    for path in tqdm(train_images):
        text2 = ax.text(0.5, 1.03, 'slice {}'.format(slice_i),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='x-large',backgroundcolor='white')
        
        # drawing img
        img = load_img(path)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = clahe.apply(img)
        im1 = ax.imshow(img, cmap='bone', animated=True)
        
        # drawing mask
        mask = id2mask(df_train[df_train['image_path']==path].iloc[0,0])*255
        if mask is not None:
            im2 = ax.imshow(mask, alpha=0.5)
        
        ims.append([im1,im2,text1,text2])
        slice_i += 1
    
    print('creating animation...')
    
    # setting animation アニメーションの設定
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.ArtistAnimation.html
    ani = animation.ArtistAnimation(fig, ims, interval=200, blit=True, repeat_delay=1000)
    ani.save(file_name+'.gif', writer='imagemagick')
    print('done!')

# Creating the gif Animation　アニメーションの作成

In [ ]:
# Specify the index number(0 ~ 273) or case and day you wish to create the animation.
# 0~273のIndex番号、または直接case-dayを指定してください。
idx = 0
CASE = df_case_day.loc[idx,'case'].astype(str)
DAY  = df_case_day.loc[idx,'day'].astype(str)

file_name = 'case'+CASE+'_day'+DAY
print(file_name)

# call the def that takes several minutes
# アニメーション作成関数呼び出し、数分かかります。
create_animation(CASE, DAY, file_name)

# display the gif animation
# 作成したアニメーションの表示。
HTML('<img src="./'+file_name+'.gif" />')

In [ ]:
# Specify the index number(0 ~ 273) or case and day you wish to create the animation.
# 0~273のIndex番号、または直接case-dayを指定してください。
CASE = '63'
DAY  = '26'

file_name = 'case'+CASE+'_day'+DAY
print(file_name)

# call the def that takes several minutes
# アニメーション作成関数呼び出し、数分かかります。
create_animation(CASE, DAY, file_name)

# display the gif animation
# 作成したアニメーションの表示。
HTML('<img src="./'+file_name+'.gif" />')